# Singular Value Decomposition (SVD) from Scratch

We manually compute the Singular Value Decomposition (SVD) of a 2×2 real matrix $( A )$.

Let:

$
A = \begin{pmatrix} 1 & 2 \\ 2 & 1 \end{pmatrix}
$

We find matrices $( U )$, $( S )$, and  $(V)$ such that:

$
A = U S V^T
$

### SVD Breakdown

1. Compute $ A^T A $
2. Compute eigenvalues and eigenvectors of  $A^T A$
3. Take square roots of eigenvalues to get singular values  $\sigma_i $
4. Construct matrix  $V$  from normalized eigenvectors (right singular vectors)
5. Compute matrix  $U$  from  $u_i = \frac{1}{\sigma_i} A v_i$
6. Reconstruct  $A $ and verify


In [1]:
import numpy as np

# Step 0: Define matrix A
A = np.array([[1, 2],
              [2, 1]], dtype=float)
print("Matrix A:")
print(A)


Matrix A:
[[1. 2.]
 [2. 1.]]


In [2]:
# Step 1: Compute A^T A
AtA = A.T @ A
print("Step 1: A^T A =")
print(AtA)


Step 1: A^T A =
[[5. 4.]
 [4. 5.]]


In [3]:
# Step 2: Compute eigenvalues and eigenvectors of A^T A

eigvals, eigvecs = np.linalg.eig(AtA)

# Sort the eigenvalues in descending order and get the corresponding indices
idx = np.argsort(eigvals)[::-1]  # argsort returns indices that would sort the array; [::-1] reverses to descending

# Reorder eigenvalues and eigenvectors using the sorted indices
eigvals = eigvals[idx]           # sorted eigenvalues (largest to smallest)
eigvecs = eigvecs[:, idx]        # reorder columns of eigenvectors accordingly

# Display the sorted eigenvalues
print("Step 2: Eigenvalues:")
print(eigvals)

# Display the corresponding (unsanitized) eigenvectors before normalization
print("Raw eigenvectors (columns of V before normalization):")
print(eigvecs)



Step 2: Eigenvalues:
[9. 1.]
Raw eigenvectors (columns of V before normalization):
[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]


In [4]:
# Step 3: Compute singular values as the square roots of the (sorted) eigenvalues
singular_values = np.sqrt(eigvals)  # Each σ_i = √λ_i, where λ_i is an eigenvalue of AᵗA

# Construct the diagonal matrix S from the singular values
S = np.diag(singular_values)  # Creates a diagonal matrix S with σ₁, σ₂, ..., on the diagonal

# Display the singular values (σ₁ ≥ σ₂ ≥ ... ≥ 0)
print("Step 3: Singular values σ:")
print(singular_values)

# Display the diagonal matrix S used in the decomposition A = U S Vᵗ
print("Step 4: Diagonal matrix S:")
print(S)



Step 3: Singular values σ:
[3. 1.]
Step 4: Diagonal matrix S:
[[3. 0.]
 [0. 1.]]


In [5]:
# Step 4: Normalize eigenvectors to construct the orthonormal matrix V (right singular vectors)
V = np.zeros_like(eigvecs)  # Initialize an empty matrix with the same shape as eigvecs

# Loop over each eigenvector to normalize it (convert to unit vector)
for i in range(eigvecs.shape[1]):
    v = eigvecs[:, i]                     # Extract the i-th eigenvector (column)
    V[:, i] = v / np.linalg.norm(v)      # Normalize it to unit length and store in V

# Display the matrix V, whose columns are orthonormal right singular vectors
print("Step 4: Normalized right singular vectors (V):")
print(V)

# Verify that V is orthogonal: VᵗV should be (close to) the identity matrix
print("Check V^T V = I?")
print(np.round(V.T @ V, decimals=4))  # Round to 4 decimal places for readability



Step 4: Normalized right singular vectors (V):
[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]
Check V^T V = I?
[[ 1. -0.]
 [-0.  1.]]


In [6]:
# Step 5: Compute the left singular vectors U using uᵢ = (1/σᵢ) * A vᵢ
U = np.zeros_like(A)  # Initialize U with the same shape as A (since U is m×m)

# Loop through each singular value/vector pair
for i in range(len(singular_values)):
    sigma = singular_values[i]                # Current singular value σᵢ
    if sigma > 1e-10:                          # Avoid division by zero or near-zero σᵢ
        U[:, i] = (A @ V[:, i]) / sigma        # Compute uᵢ = (1/σᵢ) A vᵢ

# Ensure each column of U is a unit vector (numerical safety)
for i in range(U.shape[1]):
    U[:, i] /= np.linalg.norm(U[:, i])         # Normalize uᵢ to ensure orthonormality

# Display the left singular vectors U (columns of U)
print("Step 5: Left singular vectors (U):")
print(U)

# Verify orthogonality: UᵗU should be the identity matrix
print("Check U^T U = I?")
print(np.round(U.T @ U, decimals=4))           # Round for better readability of numerical results



Step 5: Left singular vectors (U):
[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]
Check U^T U = I?
[[1. 0.]
 [0. 1.]]


In [7]:
# Step 6: Reconstruct A using the SVD components: A ≈ U S Vᵗ
A_reconstructed = U @ S @ V.T  # Matrix multiplication: U * S * Vᵗ

# Display the reconstructed matrix A from its SVD form
print("Step 6: Reconstructed A:")
print(np.round(A_reconstructed, decimals=5))  # Rounded for cleaner output

# Compute and display the difference between the original A and the reconstructed version
print("Difference A - USV^T:")
print(np.round(A - A_reconstructed, decimals=5))  # This should be close to zero if SVD was accurate



Step 6: Reconstructed A:
[[1. 2.]
 [2. 1.]]
Difference A - USV^T:
[[-0. -0.]
 [-0. -0.]]
